# List of all Dota 2 emoticons as GIFs and unicode characters

The code below generates a list of all emoticons in dota and their unicode character. You can copy & paste the character into Dota 2 to use the emoticon. It doesn't matter if you don't own the emoticon pack.

In [52]:
import os
import vdf
import vpk
import shutil 
from datetime import datetime

from PIL import Image
from IPython.display import HTML
from tqdm import tqdm


out_temp = "./.temp"
out_dir = "./gifs"
out_sizes = [32, 96]

Now let's get list of emoticons from your local Dota 2 installation. 

In [53]:
pak1 = vpk.open("C:/Program Files (x86)/Steam/steamapps/common/dota 2 beta/game/dota/pak01_dir.vpk")
data = vdf.loads(pak1['scripts/emoticons.txt'].read().decode('utf-16le'))

if not os.path.isdir(out_temp):
    os.mkdir(out_temp)
    os.mkdir(f"{out_temp}/cache")

for k, v in data['emoticons'].items():
    name = v['image_name'].replace('.', '_') + '.vtex_c'
    emoticon_path = f"panorama/images/emoticons/{name}"
    try:
        pak1[emoticon_path].save(f"{out_temp}/cache/{name}")
    except:
        print(f"Missing: {emoticon_path}")
        pass

Instead of getting emotes from local installation you can get the emoticons using [SteamCTL](https://pypi.org/project/steamctl/) ([GitHub link](https://github.com/ValvePython/steamctl)). Install it with 
```
pip install steamctl
```
And in the root of iPython notebook use these commands in Windows Terminal (it will prompt to login into steam)  to get a list of all emoticons and all `vtex_c` files:
```terminal
steamctl depot download -a 570 --vpk -n '*pak01_dir*:*emoticons.txt' -nd -o ./.temp/
steamctl depot download -a 570 --vpk -n '*pak01_dir*:*emoticons/*.vtex_c' -nd -o ./.temp/cache_vtex_c
```

But if you have dota 2 installed and succesfully run the cell above then there is no need in `steamctl`.

In [9]:
# If you use steamctl then uncomment the following line:
# data = vdf.loads(open(f'{out_temp}/emoticons.txt', 'rb').read().decode('utf-16le'))

Download VRF CLI Decompiler for windows-x64 from https://vrf.steamdb.info/ and put the path to it into the following cell. 

In [5]:
# don't mind my fancy path to Decompiler.exe
!D:/RUBBISH/ARCHIVE/VRF_CLI_Decompiler/Decompiler.exe -i "$out_temp/cache" -o "$out_temp/cache" > NUL

In [6]:
# is it bugged tho for windows ?
# Why "Decompiler.exe" ignores `-o` and sends all pictures into root of D: disk?
# Well, I have to fix it myself and move files to our desired "$out_temp/cache_png".
source = "D:\\"
dest = "D:\\LAPTOP\\Dota2Utils\\ListEmoticons\\.temp\\cache"

files = os.listdir(source)
img_extensions = (".jpg", ".gif", ".png")
for f in files:
    if f.endswith(img_extensions):
        shutil.move(source + f, dest)

In [7]:
# create ./gifs folder
if not os.path.isdir(out_dir):
    os.mkdir(out_dir)

For the next cell install ImageMagick from https://imagemagick.org/script/download.php#windows 
* I use portable `ImageMagick-7.1.0-55-portable-Q16-x64.zip` edition myself.

If you are looking at the following cell from github then I have to say: it is not showing correctly because the strings in the code have HTML elements which GitHub just eats. Download Notebook file itself instead of copypasting it from GitHub page.

In [54]:
resp = "<table><tr>" + f"<th colspan={len(out_sizes)}>preview</th><th>chr</th><th>alias</th>" * 3 + "</tr>\n"

def clear_temp_from_pngs_for_gif():
    for x in os.listdir(out_temp):
        if not x.endswith(".png"):
            continue
        os.remove(os.path.join(out_temp, x))
            
for counter, (k, v) in tqdm(enumerate(data['emoticons'].items())):
    name = f"{v['image_name'].replace('.', '_')}.png"
    try:
        f = open(f"{out_temp}/cache/{name}", 'rb')
    except IOError:
        print('reeee')
        continue
    out_paths = [f"{out_dir}/{int(k):0>3d}_{size:d}.gif" for size in out_sizes]

    # if a gif doesn't exist generate it
    if not all(map(os.path.isfile, out_paths)):  
        clear_temp_from_pngs_for_gif()

        # split the png sequence in seperate files                              
        im = Image.open(f)
        for out_path, size in zip(out_paths, out_sizes):
            for i in range(im.size[0] // 32):
                im.crop((i * 32, 0, (i + 1) * 32, 32))\
                .resize((size, size), Image.NEAREST)\
                .save(f"{out_temp}/{i:0>3d}.png")
            # combines the sequence images into a gif using ImageMagick
            !D:/RUBBISH/PRO/ImageMagick/convert -loop 0 -delay 10 -alpha set -dispose background "$out_temp/*.png" "$out_path"
    
    # generate table cells for emoticon
    if counter % 3 == 0:
        resp += "<tr>\n"    
    for path in out_paths:
        resp += f"<td><img src='{path}'></td>\n"
    resp += f"<td>{chr(0xE000 + int(k)):s}</td>\n<td>`:{v['aliases']['0']:s}:`</td>\n"
    if counter % 3 == 2:
        resp += "</tr>\n"

clear_temp_from_pngs_for_gif()
if (counter - 1) % 3 != 2:
    resp += "</tr>"
resp += "</table>"

321it [00:00, 5029.16it/s]


In [56]:
with open('./README.md', 'r', encoding='utf-8') as file:
    file_lines = file.readlines()
    old_lines = []
    for line in file_lines:
        if not line.startswith('### Table of Dota 2 Emoticons'):
            old_lines.append(line)
        else:
            old_lines.append('### Table of Dota 2 Emoticons\n')
            old_lines.append(f'Last Updated: {datetime.today().strftime("LAST UPDATED: %d/%B/%Y")} | Amount of emoticons found: {counter + 1}\n')
            break

with open('./README.md', 'w', encoding='utf-8') as file:
    file.writelines(old_lines)
    
with open('./README.md', 'a', encoding='utf-8') as file:
    file.write(resp)

In [16]:
print(f'Amount of emoticons found: {counter + 1}')
HTML(resp)